# Visualising data with Autoencoders and Variational Autoencoders

### 1. FMNIST preparation


### 2. Autoencoders 
   - Model explanation (encoding - decoding)
   - Zastosowania Autoencoderów
   - Encoding to 2 dimensions and plotting (tak jest przy wielowymiarowych danych, do FMNISTA to nie wiem jeszcze do końca co my tam mamy wizualizować, chyba że inne dane)




### 3. Variational Autoencoder 
   - Model explanation 
   - Mathematical formulas for noise injection 
   - Encoding to 2 dimensions and plotting
   - Probkowanie fantazji sieci


### 4. Porównanie dwóch metod
